In [1]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Dropout
from keras.layers import Flatten
from google.colab import drive

In [3]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale = 1./255)
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [4]:
x_train = train_datagen.flow_from_directory('gdrive/My Drive/training_set/', target_size=(64,64), batch_size=300, class_mode='categorical',  color_mode = "grayscale")

Found 15759 images belonging to 9 classes.


In [5]:
x_train.class_indices

{'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6, 'H': 7, 'I': 8}

In [6]:
x_test = test_datagen.flow_from_directory('gdrive/My Drive/test_set/', target_size=(64,64), batch_size=300, class_mode='categorical',  color_mode = "grayscale")

Found 1310 images belonging to 9 classes.


In [7]:
model = Sequential()

In [8]:
model.add(Convolution2D(32, (3,3), input_shape=(64,64,1), activation = 'relu'))
#no. of feature detectors, size of featuredetector, image size, activation function

In [9]:
model.add(MaxPooling2D(pool_size=(2,2)))

In [10]:
model.add(Dropout(0.25))

In [11]:
model.add(Convolution2D(64, (3,3), input_shape=(64,64,1), activation = 'relu'))
#no. of feature detectors, size of featuredetector, image size, activation function

In [12]:
model.add(MaxPooling2D(pool_size=(2,2)))

In [13]:
model.add(Dropout(0.25))

In [14]:
model.add(Convolution2D(128, (3,3), input_shape=(64,64,1), activation = 'relu'))
#no. of feature detectors, size of featuredetector, image size, activation function

In [15]:
model.add(MaxPooling2D(pool_size=(2,2)))

In [16]:
model.add(Dropout(0.25))

In [17]:
model.add(Flatten())

In [18]:
model.add(Dense(units=512, activation='relu'))

In [19]:
model.add(Dense(units=9, activation='softmax'))

In [20]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [21]:
model.fit_generator(x_train, steps_per_epoch=24, epochs=10, validation_data=x_test, validation_steps=40)

#steps_per_epoch = no. of train images//batch size

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
24/24 [==============================] - 3410s 142s/step - loss: 1.5471 - accuracy: 0.4363 - val_loss: 0.5670 - val_accuracy: 0.8504
Epoch 2/10
24/24 [==============================] - 1354s 56s/step - loss: 0.2721 - accuracy: 0.9119
Epoch 3/10
24/24 [==============================] - 722s 30s/step - loss: 0.1048 - accuracy: 0.9668
Epoch 4/10
24/24 [==============================] - 382s 16s/step - loss: 0.0624 - accuracy: 0.9766
Epoch 5/10
24/24 [==============================] - 211s 9s/step - loss: 0.0432 - accuracy: 0.9867
Epoch 6/10
24/24 [==============================] - 127s 5s/step - loss: 0.0400 - accuracy: 0.9873
Epoch 7/10
24/24 [==============================] - 83s 3s/step - loss: 0.0342 - accuracy: 0.9867
Epoch 8/10
24/24 [==============================] - 64s 3s/step - loss: 0.0205 - accuracy: 0.9933
Epoch 9/10
24/24 [==============================] - 54s 2s/step - loss: 0.0150 - accuracy: 0.9953
Epoch 10/10
24/24 [==============================] - 57s 2s/ste

In [22]:
model.save('aslpng1.h5')

In [23]:
from keras.models import load_model
import numpy as np
import cv2

In [24]:
from keras.models import load_model
model=load_model('aslpng1.h5')

In [25]:
from skimage.transform import resize
def detect(frame):
        img = resize(frame,(64,64,1))
        img = np.expand_dims(img,axis=0)
        if(np.max(img)>1):
            img = img/255.0
        prediction = model.predict(img)
        print(prediction)
        prediction = model.predict_classes(img)
        print(prediction)

In [34]:
frame=cv2.imread(r"gdrive/My Drive/training_set/F/1.png")
data = detect(frame)

[[6.32418039e-16 4.34410715e-15 5.48170775e-13 1.10858114e-10
  4.96806868e-18 9.99970913e-01 5.03634508e-17 6.03531654e-22
  2.91262404e-05]]
[5]


/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [37]:
import numpy as np
import cv2
import os
from keras.models import load_model
from flask import Flask, render_template, Response
import tensorflow as tf
from gtts import gTTS #to convert text to speech
global graph
global writer
from skimage.transform import resize

In [39]:
graph = tf.Graph()
writer = None

model = load_model('aslpng1.h5')

vals = ['A', 'B','C','D','E','F','G','H','I']

app = Flask(__name__)

print("[INFO] accessing video stream...")
vs = cv2.VideoCapture(0) #triggers the local camera

pred=""


[INFO] accessing video stream...


In [40]:
def detect(frame):
        img = resize(frame,(64,64,1))
        img = np.expand_dims(img,axis=0)
        if(np.max(img)>1):
            img = img/255.0
        with graph.as_default():
                prediction = model.predict_classes(img)
        print(prediction)
        pred=vals[prediction[0]]
        print(pred)
        return pred


In [41]:
@app.route('/')
def index():
    return render_template('index.html')


In [42]:
def gen():
        while True:
            # read the next frame from the file
            (grabbed, frame) = vs.read()
            
            # if the frame was not grabbed, then we have reached the end
            # of the stream
            if not grabbed:
                break
            
            data = detect(frame)
            # output frame
            text = "It indicates "+data
            cv2.putText(frame, text, (10, frame.shape[0] - 25),cv2.FONT_HERSHEY_SIMPLEX, 0.85, (0, 0, 255), 4)
            cv2.imwrite("1.jpg",frame)

            #converts text to speech and plays the audio
            speech = gTTS(text = data, lang = 'en', slow = False)
            speech.save("text.mp3")
            os.system("start text.mp3")
            
            key = cv2.waitKey(1) & 0xFF
            # if the `q` key was pressed, break from the loop
            if key == ord("q"):
                break
            fourcc = cv2.VideoWriter_fourcc(*"MJPG")
            writer = cv2.VideoWriter(r"output.avi", fourcc, 25,(frame.shape[1], frame.shape[0]), True)

            (flag, encodedImage) = cv2.imencode(".jpg", frame)
            yield (b'--frame\r\n' b'Content-Type: image/jpeg\r\n\r\n' + 
                                bytearray(encodedImage) + b'\r\n')

In [44]:
@app.route('/video_feed')
def video_feed():
    return Response(gen(),
                    mimetype='multipart/x-mixed-replace; boundary=frame')

if __name__ == '__main__':
    app.run(host='0.0.0.0', debug=True)

AssertionError: ignored